# Lesson 04: Concurrency & Parallelism

## 0. Waiting

No-one wants to wait! In programming, waiting == lost time and performance.

1. CPU bound (waiting on computation)
    - use parallelism
    - e.g. iterate through very large file & transform to different format
2. IO bound (waiting for hard disk or network)
    - sending http request
    - writing data to disk

---

## 1. CPU Bound (Parallelism)

Compared to concurrency, parallelism is easier to use, and is _usually_ easier to think about and design. Unfortunately, it's less commonly needed, as CPU bound processes usually only occur in certain scenarios, aka

> **Big Data processing**

### Data processing example

We can explore this through a data-processing scenario: generating large JSON lines to write to file

- This could be a transform of original files -> a more suitable format for insights
- Manipulating database dumps (postgres/ES)

---

### Example 1: Threading

*do some main work, and some work on the side!*

1. We need to generate a _very_ large NDJSON file (newline-delimited JSON). For simplicities sake, all lines are readable/same schema etc.

    ```json
    {"a": "B"}
    {"a": "C"}
    ```

    1. There are many language and OS-level optimisations around doing the _exact_ same thing, like performing the same calculation over the same file line data. This means that we have to randomise the values in order to make a good test file.


2. use `faker`! 

In [1]:
from faker import (Faker, providers)
F = Faker()
F.add_provider(providers.misc)
F.add_provider(providers.geo)

In [2]:
def fkr_n(fkr, n):
    return [fkr() for _ in range(n)]

def gen_movie(f=F, indent=None):
    "Generates a fake movie listing"

    return json.dumps(
        {
            "titleId":         f.uuid4(),
            "ordering":        f.random_int(),
            "title":           f.catch_phrase(),
            "region":          f.locale(),
            "language":        f.language_name(),
            "types":           fkr_n(f.name, 5),
            "attributes":      fkr_n(f.name, 5),
            "isOriginalTitle": f.boolean(),
            "tconst":          f.uuid4(),
            "titleType":       f.domain_name(),
            "primaryTitle":    f.catch_phrase(),
            "originalTitle":   ":".join([f.company(), f.catch_phrase()]),
            "isAdult":         f.boolean(),
            "startYear":       f.date(),
            "endYear":         f.year(),
            "runtimeMinutes":  f.random_int(),
            "genres":          fkr_n(f.country, 5),
            "tconst":          f.hex_color(),
            "directors":       fkr_n(f.name, 2),
            "writers":         fkr_n(f.name, 15),
            "actors":          fkr_n(f.name, 50),
        },
        indent=indent
    )


In [3]:
import json
from pygments import highlight, lexers, formatters

def pretty_print_json(j):
    print(highlight(j, lexers.JsonLexer(), formatters.TerminalFormatter()))

In [4]:
pretty_print_json(gen_movie(indent=2))

{
  "titleId": "393638b8-1d8f-4b3b-a170-f287192efe70",
  "ordering": 8916,
  "title": "Managed scalable instruction set",
  "region": "fy_DE",
  "language": "Galician",
  "types": [
    "Willie Baker",
    "Derek Shannon",
    "Heather Mcintyre",
    "Bradley Williams",
    "Sherry Campbell"
  ],
  "attributes": [
    "Terry Anderson",
    "Mary Conway",
    "Luke Evans",
    "Anthony Howard",
    "Dylan King"
  ],
  "isOriginalTitle": true,
  "tconst": "#7b1fee",
  "titleType": "velasquez.com",
  "primaryTitle": "Organized client-server synergy",
  "originalTitle": "Johnson, Wood and Garcia:Exclusive disintermediate hardware",
  "isAdult": true,
  "startYear": "1981-07-22",
  "endYear": "1980",
  "runtimeMinutes": 3617,
  "genres": [
    "Congo",
    "Morocco",
    "Dominican Republic",
    "Netherlands Antilles",
    "Aruba"
  ],
  "directors": [
    "Joseph Grimes",
    "Alexis Bauer"
  ],
  "writers": [
    "John Flores",
    "Jeffrey Acevedo",
    "Traci Wilkins",
    "Rodney Rand

Woohoo! Now we just need to write this to a file

In [5]:
class MovieTable:
    def records(fpath, n_records=10):
        "Writes random movies to a file"

        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for _ in range(n_records):
                print(gen_movie(), file=ostream)

In [6]:
MovieTable.records("/tmp/movies.ndjson", 200)

Writing 200 records to /tmp/movies.ndjson


Now let's introduce another process - a monitor that will tell us how much CPU we are currently using

> Use psutil

In [7]:
import psutil
def cpu(*args):
    print("\t".join(map(str, psutil.cpu_percent(percpu=True))))

cpu()

0.0	0.0	0.0	0.0	0.0	0.0


Now, let's experiment with running some sort of loop, and _also_ running our CPU checker.

Let's try `threading`! This is the most basic entrypoint into async programming in python

In [8]:
import threading
import time

def loop_with_timer(n):
    timer = threading.Timer(1, cpu)
    timer.start()

    # Actually do stuff here
    for i in range(n):
        print(i)
        time.sleep(1)

    timer.cancel()

# If we run this, we should see a CPU check every second?
loop_with_timer(5)

0
1
1.8	1.8	1.8	3.7	5.5	0.9
2
3
4


Huh? The threading. Timer only printed once?!

- The "interval" of the timer is more like a "delay"   
- from the docs:
    ```
    "Create a timer that will run function with arguments args and keyword arguments kwargs, 
    after interval seconds have passed."
    ```

In [9]:
# Shout out -> https://stackoverflow.com/a/48741004
class RepeatTimer(threading.Timer):
    def run(self):
        while not self.finished.wait(self.interval):
            self.function(*self.args, **self.kwargs)

def loop_with_timer(n):
    timer = RepeatTimer(1, cpu)
    timer.start()

    for i in range(n):
        print(i)
        time.sleep(1)

    timer.cancel()

loop_with_timer(5)

0
2.4	2.4	1.8	2.6	4.0	4.4
1
2
0.0	0.0	0.0	0.0	0.0	0.0
3
2.0	1.0	1.0	2.0	5.1	2.0
0.0	2.0	0.0	1.0	2.0	0.0
4
2.0	1.0	1.0	5.1	2.0	2.0


Hmmm, there's a downside here -   
In every function that we want to track CPU usage for, we have to add all of this threading code which is pretty icky

However - we could make a decorator function that would allow anything to be timed just by adding a `@decoration`

In [10]:
import functools

def with_cpu(func):
    'Looks gross, but you only have to write it once!'

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Start the timer
        timer = RepeatTimer(1, cpu, args=None, kwargs=None)
        timer.start()

        # Do thing
        result = func(*args, **kwargs)

        # Stop timer
        timer.cancel()
        print("wrapper fin!")
        return result
    return wrapper

Now, we can just add `@with_cpu` to any function that we need to investigate

In [11]:
@with_cpu
def poc():
    for i in range(5):
        print(i)
        time.sleep(1)

In [12]:
poc()

0
12.6	4.3	0.9	3.5	4.3	0.9

22.0	2.0	1.0	1.0	7.1	3.0

3
0.0	1.0	1.0	0.0	2.0	0.0
4
2.0	2.0	0.0	2.0	6.1	3.0
wrapper fin!
1.0	0.0	0.0	0.0	2.0	0.0


Let's add the decorator to our `MovieTable.records` method

In [13]:
class MovieTable:

    @with_cpu
    def records(fpath, n_records=10):
        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for _ in range(n_records):
                print(gen_movie(), file=ostream)

In [14]:
MovieTable.records("/tmp/movies.ndjson", 500)

Writing 500 records to /tmp/movies.ndjson
8.8	10.1	17.5	67.3	9.8	2.7
1.0	0.0	0.0	100.0	2.0	1.0
1.0	0.0	2.0	99.0	5.9	2.0
0.0	0.0	0.0	100.0	1.0	0.0
1.0	1.0	2.8	100.0	5.7	2.9
1.0	1.0	0.0	100.0	1.0	0.0
wrapper fin!


Nice!

- Now we're running some code, an running another looping bit of code in a thread on the side.
- We can see that we are currently using 1 core, and that core is running at 100%

### What about `async`?

1. `asyncio` give more control over when threads switch
2. there is more code to write

In [15]:
import json
import asyncio
from dataclasses import dataclass

import nest_asyncio
nest_asyncio.apply()

import psutil

@dataclass
class Timer:
    f:        object
    id:       int  = 1
    sentinel: bool = False

    def task(self):
        "Starts the timer"
        return asyncio.create_task(self._run())

    async def stop(self):
        "Stops the timer. This is the magic! Cancelling/force-halting things in async is not nice"
        self.sentinel = True

    async def _run(self):
        while not self.sentinel:
            self.f(self.id)
            await asyncio.sleep(1)

In [16]:
async def run_with_timer(f: object, t: Timer):
    t.task()       # Start timer
    await f        # Do thing
    await t.stop() # Stop timer, otherwise it will run forever
    print("async timer fin!")

Now that we have our async timer, we also have to write an async version of AMovieTable

In [17]:
class AMovieTable:
    async def records(fpath, n_records=10):
        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for _ in range(n_records):
                print(gen_movie(), file=ostream) # Do work
                await asyncio.sleep(0)           # Yield control to another thread

In [18]:
asyncio.run(
    run_with_timer(
        AMovieTable.records("/tmp/movies.ndjson", 500),
        Timer(cpu)
    )
)

Writing 500 records to /tmp/movies.ndjson
25.0	6.9	7.4	18.5	25.0	30.0
3.0	1.9	1.0	2.0	100.0	1.0
2.9	0.0	2.0	0.0	100.0	0.0
2.9	1.0	1.0	0.0	100.0	1.0
2.9	0.0	1.0	2.0	100.0	0.0
4.8	0.0	2.9	0.0	99.0	1.0
async timer fin!


Cool, cool. How could we do the same thing, but with a decorator?

In [19]:
def awith_cpu(func):
    @functools.wraps(func)
    async def wrapped(*args, **kwargs):
        t = Timer(cpu)
        t.task()

        await func(*args, **kwargs)

        await t.stop()
        print("async dec fin!")
    return wrapped

In [20]:
class AMovieTable:
    @awith_cpu
    async def records(fpath, n_records=10):
        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for _ in range(n_records):
                print(gen_movie(), file=ostream) # Do work
                await asyncio.sleep(0)           # Yield control to another thread

# This is much neater than the previous version
# asyncio.run(
#     run_with_timer(
#         AMovieTable.records("/tmp/movies.ndjson", 500),
#         Timer(cpu)
#     )
# )
asyncio.run(
    AMovieTable.records("/tmp/movies.ndjson", 500)
)

Writing 500 records to /tmp/movies.ndjson
6.4	7.0	3.6	4.6	89.5	4.5
1.0	3.1	1.9	2.0	0.0	100.0
0.0	4.0	1.0	0.0	1.0	99.0
1.9	3.0	1.0	0.0	1.0	100.0
0.0	0.0	0.0	1.0	1.9	100.0
1.0	1.0	2.9	1.0	2.0	100.0
async dec fin!


In [21]:
!wc -l "/tmp/movies.ndjson" && ls -alh "/tmp/movies.ndjson" && head -c 100 "/tmp/movies.ndjson"

500 /tmp/movies.ndjson
-rw-r--r-- 1 jovyan users 943K Dec  1 13:34 /tmp/movies.ndjson
{"titleId": "d2670f25-c9e3-4bce-8a4b-060953066d29", "ordering": 2989, "title": "Assimilated tangible

## Faster? -> multiprocessing

Now that we have saturated the usage on a single core, let's run this on more cores

In [22]:
from multiprocessing import Process

# No point being fancy about setting up this list
procs = []
for fpath in ["/tmp/movies.ndjson", "/tmp/movies2.ndjson"]:
    p = Process(target=MovieTable.records, args=(fpath, 500))
    p.start()
    procs.append(p)

while True:
    if not any(p.is_alive() for p in procs):
        print("process fin!")
        break

Writing 500 records to /tmp/movies.ndjsonWriting 500 records to /tmp/movies2.ndjson

2.6	53.7	3.7	53.4	19.8	70.3
3.1	54.2	3.7	53.6	19.8	70.5
1.0	100.0	1.0	99.0	0.0	100.0
1.0	100.0	1.0	99.0	1.9	100.0
3.9	100.0	2.0	100.0	5.9	100.0
3.0	100.0	2.0	100.0	4.0	100.0
0.0	100.0	1.0	68.0	33.7	99.0
1.0	99.0	1.0	67.0	34.6	99.0
5.0	99.0	3.0	4.9	100.0	100.0
5.0	100.0	2.9	5.8	100.0	100.0
wrapper fin!
wrapper fin!
process fin!


In [23]:
from multiprocessing import Pool

def run_async_pool(args):
    asyncio.run(AMovieTable.records(*args))


with Pool(2) as p:
    p.map(run_async_pool, [("/tmp/movies.ndjson", 500), ("/tmp/movies2.ndjson", 500)])

Writing 500 records to /tmp/movies2.ndjsonWriting 500 records to /tmp/movies.ndjson

3.1	84.3	2.7	59.6	33.5	89.2
3.1	84.3	2.7	59.6	33.5	89.2
2.0	99.0	3.9	1.0	99.0	5.0
1.9	99.0	3.9	1.0	99.0	4.9
1.0	100.0	0.0	2.0	100.0	2.0
1.0	100.0	0.0	1.9	100.0	2.9
3.9	100.0	1.9	3.0	100.0	5.8
3.9	100.0	2.9	4.0	99.0	4.9
1.9	100.0	1.0	2.9	99.0	0.0
1.9	100.0	1.9	1.9	100.0	1.0
5.0	99.0	3.9	2.0	100.0	3.9
5.0	99.0	2.0	2.0	100.0	3.9
async dec fin!
async dec fin!


### Logging

Don't forget about logging! As you can see, it's hard to tell what thread is producing any particular log line

- pass an ID/name/any metadata that you might need to the method in question, and ensure that it logs using it

In [24]:
def awith_cpu(func):
    @functools.wraps(func)
    async def wrapped(*args, **kwargs):
        t = Timer(cpu, args[0]) # <---
        t.task()

        await func(*args, **kwargs)

        await t.stop()
        print("fin!")
    return wrapped

class AMovieTable:
    @awith_cpu
    async def records(i, fpath, n_records=10):
        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for _ in range(n_records):
                print(gen_movie(), file=ostream) # Do work
                await asyncio.sleep(0)           # Yield control to another thread


def cpu(*args):
    print(" - ".join([str(args), "\t".join(map(str, psutil.cpu_percent(percpu=True)))]), flush=True)

def run_multiproc(nprocs):
    fpaths = []
    for i in range(nprocs):
        fpaths.append((i, f'/tmp/movies_{i}.ndjson', 500))

    with Pool(nprocs) as p:
        p.map(run_async_pool, fpaths)


In [25]:
run_multiproc(2)

Writing 500 records to /tmp/movies_0.ndjsonWriting 500 records to /tmp/movies_1.ndjson

(0,) - 3.4	90.5	2.4	32.7	63.9	48.4(1,) - 3.4	90.5	2.4	32.7	63.9	48.4

(0,) - 16.5	0.0	99.0	85.4	2.0	1.0(1,) - 16.5	0.0	99.0	85.4	2.0	1.0

(0,) - 5.9	2.0	100.0	99.0	1.0	2.9
(1,) - 5.9	1.9	100.0	99.0	1.0	2.9
(0,) - 5.8	1.9	100.0	100.0	2.0	1.0
(1,) - 5.8	1.9	100.0	100.0	4.0	1.9
(0,) - 6.9	2.9	100.0	100.0	2.0	1.9
(1,) - 6.9	2.9	100.0	100.0	1.0	2.0
(0,) - 1.9	1.9	100.0	99.0	1.9	1.0
(1,) - 2.9	1.9	99.0	99.0	1.0	1.0
fin!
fin!


## Thoughts on the GIL

- Python process == 1 thread is analagous to microservices architecture (lambda)
- This is easy to reason about and to design
    - For a single thread, write code that ensures that the usage is saturated
    - If you need parallel, simply take this same code and run it on another core (multiprocessing)

The best way to demonstrate this is to take the same approach, and run it across every CPU thread

In [26]:
run_multiproc(os.cpu_count())

Writing 500 records to /tmp/movies_2.ndjsonWriting 500 records to /tmp/movies_3.ndjsonWriting 500 records to /tmp/movies_0.ndjsonWriting 500 records to /tmp/movies_1.ndjson

Writing 500 records to /tmp/movies_4.ndjson



Writing 500 records to /tmp/movies_5.ndjson(2,) - 4.9	61.6	34.1	53.6	43.9	33.4(1,) - 4.9	61.6	34.1	53.6	43.9	33.4
(0,) - 4.9	61.6	34.1	53.6	43.9	33.4

(3,) - 5.0	61.7	34.1	53.6	44.0	33.4
(5,) - 5.0	61.7	34.1	53.6	44.0	33.4(4,) - 5.1	61.7	34.1	53.6	44.0	33.4

(2,) - 99.0	98.0	99.0	99.0	99.0	98.0
(1,) - 99.0	98.1	99.0	99.0	98.1	98.1(0,) - 99.0	98.1	99.0	99.0	98.1	98.1

(3,) - 99.0	98.1	99.0	100.0	99.0	98.1
(5,) - 99.0	98.1	99.0	100.0	99.0	99.0
(4,) - 99.0	98.1	99.0	100.0	99.1	98.1
(2,) - 98.0	99.0	99.0	100.0	99.0	99.0
(1,) - 98.0	99.0	99.0	100.0	99.0	99.0
(0,) - 98.1	98.1	99.1	100.0	99.0	99.1(3,) - 98.1	98.1	99.0	100.0	99.0	99.0

(5,) - 98.1	98.1	98.2	100.0	99.1	97.2(4,) - 98.1	99.0	98.1	100.0	99.0	98.1

(2,) - 97.9	98.9	97.9	97.9	98.9	97.9
(1,) - 97.9	98.9	97.9	97.9	97.

We have now achieved **saturation**

![Saturation!](tumblr_p47wmu8e3P1w1x3muo1_500.jpeg)

## 2. Communication

If possible, just don't communicate! Think about whether or not you _need_ to pass a result or value around

But if you need to,

### Example 2. Consider files

You can write to IO in async functions

In [27]:
from io import StringIO

async def a(ostream, chr):
    ostream.write(chr)

s = StringIO()
asyncio.run(asyncio.gather(
    a(s, 'a'),
    a(s, 'b')
))

s.seek(0)
print(s.readlines())

['ab']


You can also read from them

In [28]:
s = StringIO('''asdfsdf
234234
oeairjl
3049568
er;ij
''')

In [29]:
async def b(istream, i):
    l = istream.readline()
    await asyncio.sleep(0)
    print(i, l)

asyncio.run(asyncio.gather(b(s, 1), b(s, 2), b(s, 3)))

1 asdfsdf

2 234234

3 oeairjl



[None, None, None]

You might notice that `asyncio.gather` returns the results of the async tasks.

Unfortunately, using `gather` with _many_ async tasks results in memory blowout, so using gather is usually not advised unless you know the number/size of results is low

In [30]:
!wc -l /tmp/movies.ndjson

500 /tmp/movies.ndjson


In [31]:
q = asyncio.Queue(2)

class Sentinel(): pass

async def producer(q, istream, sentinel=Sentinel):
    for line in istream:
        await q.put(line)
    await q.put(sentinel)

async def consumer(q, ostream, sentinel=Sentinel):
    while True:
        d = await q.get()
        if d == sentinel:
            await q.put(sentinel)
            return
        data = json.loads(d)["title"]
        await asyncio.sleep(0)
        print(data, file=ostream)


def run_queue(qsize):
    q = asyncio.Queue(qsize)
    with open("/tmp/movies.ndjson") as istream, open("/tmp/movie_titles.txt", "w") as ostream:
        asyncio.run(asyncio.gather(
            producer(q, istream),
            consumer(q, ostream),
        ))


In [32]:
%timeit run_queue(2)
%timeit run_queue(20)
%timeit run_queue(100)

34.2 ms ± 600 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
37 ms ± 6.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
35.9 ms ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
!wc -l /tmp/movie_titles.txt /tmp/movies.ndjson && head -n 3 /tmp/movie_titles.txt

   500 /tmp/movie_titles.txt
   500 /tmp/movies.ndjson
  1000 total
Self-enabling needs-based synergy
Robust heuristic intranet
Organic motivating info-mediaries


## Example 3: Extending the standard lib

This should always be your first port of call when prototyping something in Python. Python has a pattern of not implementing "framework" features and patterns, but it gives you the tools to start your own.

*Remember - every python async framework is written in python*

For this example, I tried to tackle the problem of controlling async requests by a "rate limit".

I haven't gone whole-hog and implemented the queue producer, I've just used files instead. The key part to look at would be the different ThrottledQueue instantiations, and the producer/consumer combos that use them

In [86]:
class ThrottledQueue(asyncio.Queue):
    "subclass asyncio.Queue i.e. import all behaviour"

    def __init__(self, per_second, maxsize=0, *, loop=None, i=0):
        "Set up some extra vars and then call the original init"

        self.i = i
        self.per_second = per_second
        self.last_get = time.time() # this is the fastest way... I think?
        super(ThrottledQueue, self).__init__(maxsize=maxsize, loop=loop)

    async def get(self):
        "Throttles, sleep "

        elapsed = time.time() - self.last_get
        sleep_time = self.per_second - elapsed
        print(self.i, '- times', f'{elapsed:.5f}', f'{sleep_time:.5f}', '- sizes', self.qsize(), f'{self.qsize() / self.maxsize:.5f}')
        await asyncio.sleep(max(0, sleep_time)) # Make sure we wait at least 0 seconds

        result = await super(ThrottledQueue, self).get()

        self.last_get = time.time()
        return result

In [76]:
def run_queue(per_second, qsize):
    q = ThrottledQueue(per_second, qsize)
    with open("/tmp/movies.ndjson") as istream, open("/tmp/movie_titles.txt", "w") as ostream:
        asyncio.run(asyncio.gather(
            producer(q, istream), # <-- queue in
            consumer(q, ostream), # <-- queue out
        ))

In [78]:
run_queue(0.1, 20)
print('--')
run_queue(1, 2)

0 - times 0.00429 0.09571 - sizes 8 0.40000
0 - times 0.00025 0.09975 - sizes 7 0.35000
0 - times 0.00010 0.09990 - sizes 6 0.30000
0 - times 0.00012 0.09988 - sizes 5 0.25000
0 - times 0.00024 0.09976 - sizes 4 0.20000
0 - times 0.00013 0.09987 - sizes 3 0.15000
0 - times 0.00022 0.09978 - sizes 2 0.10000
0 - times 0.00011 0.09989 - sizes 1 0.05000
--
0 - times 0.00088 0.99912 - sizes 2 1.00000
0 - times 0.00014 0.99986 - sizes 2 1.00000
0 - times 0.00023 0.99977 - sizes 2 1.00000
0 - times 0.00017 0.99983 - sizes 2 1.00000
0 - times 0.00018 0.99982 - sizes 2 1.00000
0 - times 0.00020 0.99980 - sizes 2 1.00000
0 - times 0.00022 0.99978 - sizes 2 1.00000
0 - times 0.00009 0.99991 - sizes 1 0.50000


Okay... how about >1 queue?

In [87]:
def run_queues(per_second, per_second2, qsize):
    q = ThrottledQueue(per_second, qsize, i=1)
    q2 = ThrottledQueue(per_second2, qsize, i=2)
    with open("/tmp/movies.ndjson") as istream, open("/tmp/movies_2.ndjson") as istream2, open("/tmp/movie_titles.txt", "w") as ostream:
        asyncio.run(asyncio.gather(
            producer(q, istream), producer(q2, istream2), # <-- q1 & q2 in
            consumer(q, ostream), consumer(q2, ostream),  # <-- q1 & q2 out
        ))

In [89]:
run_queues(1, 0.1, 20)

1 - times 0.00479 0.99521 - sizes 8 0.40000
2 - times 0.00533 0.09467 - sizes 20 1.00000
2 - times 0.00019 0.09981 - sizes 20 1.00000
2 - times 0.00012 0.09988 - sizes 20 1.00000
2 - times 0.00018 0.09982 - sizes 20 1.00000
2 - times 0.00047 0.09953 - sizes 20 1.00000
2 - times 0.00030 0.09970 - sizes 20 1.00000
2 - times 0.00023 0.09977 - sizes 20 1.00000
2 - times 0.00017 0.09983 - sizes 20 1.00000
2 - times 0.00016 0.09984 - sizes 20 1.00000
2 - times 0.00016 0.09984 - sizes 20 1.00000
1 - times 0.00030 0.99970 - sizes 7 0.35000
2 - times 0.00038 0.09962 - sizes 20 1.00000
2 - times 0.00018 0.09982 - sizes 20 1.00000
2 - times 0.00035 0.09965 - sizes 20 1.00000
2 - times 0.00035 0.09965 - sizes 20 1.00000
2 - times 0.00029 0.09971 - sizes 20 1.00000
2 - times 0.00013 0.09987 - sizes 20 1.00000
2 - times 0.00026 0.09974 - sizes 20 1.00000
2 - times 0.00041 0.09959 - sizes 20 1.00000
2 - times 0.00018 0.09982 - sizes 20 1.00000
2 - times 0.00018 0.09982 - sizes 20 1.00000
1 - times 0.